In [1]:
import csv

snomeds = []
# open the CSV file in read mode
with open('../../SNOMEDCT_CORE_SUBSET_202211/SNOMEDCT_CORE_SUBSET_202211.txt', 'r') as csvfile:
    # create a CSV reader object
    csvreader = csv.reader(csvfile, delimiter="|")
    
    # get the header row and create a dictionary of column indexes
    header = next(csvreader)
    indexes = {header[i]: i for i in range(len(header))}

    # loop over the remaining rows and create a dictionary for each row
    for row in csvreader:
        snomed = {}
        for key, index in indexes.items():
            snomed[key] = row[index]
        snomed["text"] = snomed["SNOMED_FSN"]
        snomeds.append(snomed)

len(snomeds)

6667

In [36]:
import sqlite3
import re
conn = sqlite3.connect(":memory:")

cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS vocab")
cur.execute("CREATE VIRTUAL TABLE IF NOT EXISTS vocab USING fts5(code, display, related, common_test_rank, units, tokenize = \"porter unicode61 remove_diacritics 1 separators '-_'\")")
cur.execute("INSERT into vocab(vocab, rank) values('rank', ' bm25(0, 10, 0, 0)')")

for r in snomeds:
    code = r['SNOMED_CID']
    display = r['text']
    related = ""
    common_test_rank = 0
    units = ""
    cur.execute("INSERT INTO vocab (code, display, related, common_test_rank, units) VALUES (?, ?, ?, ?, ?)", (code, display, related, common_test_rank, units));

def search_loinc(query, limit = 100, skip = 0):
  print(query)
  cur.execute("select * from (SELECT *, rank FROM vocab WHERE vocab MATCH ? ORDER BY rank limit ? offset ?) order by common_test_rank", (" OR ".join(query.replace("-", " ").split(" ")), limit, skip,))
  results = cur.fetchall()
  print(f"---\nSearch results for '{query}':")
  for row in results:
    print(f"{row[0]}, {row[1]} ctr {row[3]} {row[-1]}")

search_loinc("post-partum depression")
conn.commit()
conn.close()

post-partum depression
---
Search results for 'post-partum depression':
27151001, Post-laminectomy syndrome (disorder) ctr 0 -12.274184592849156
314116003, Post infarct angina (disorder) ctr 0 -12.274184592849156
31097004, Post poliomyelitis syndrome (disorder) ctr 0 -12.274184592849156
239783001, Post-infective arthritis (disorder) ctr 0 -12.274184592849156
238699007, Post-inflammatory hyperpigmentation (disorder) ctr 0 -12.274184592849156
236664000, Post-micturition incontinence (finding) ctr 0 -12.274184592849156
202725007, Lumbar post-laminectomy syndrome (disorder) ctr 0 -12.126262902026289
269406001, Post-traumatic wound infection (disorder) ctr 0 -12.126262902026289
202723000, Cervical post-laminectomy syndrome (disorder) ctr 0 -12.126262902026289
373108000, Post percutaneous transluminal coronary angioplasty (finding) ctr 0 -11.981864096311027
313182004, Chronic post-traumatic stress disorder (disorder) ctr 0 -11.981864096311027
226007004, Post-surgical wound care (regime/thera

In [ ]:
"post-c".replace("post", "post ")

In [ ]:
from txtai.embeddings import Embeddings

embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2", "content": True})

embeddings.index([(s["SNOMED_CID"], s, None) for s in snomeds])


In [ ]:
for r in embeddings.search("SELECT id, text, score FROM txtai WHERE similar('major depressive disorder, recurrent, severe')  limit 100"):
    if r['score'] >= .65:
        print(r['id'], r['text'])

In [ ]:
import nltk
token_counts = {}
from nltk.stem import PorterStemmer
ps = PorterStemmer()

for r in snomeds:
    r["textTokens"] = set([ps.stem(v.lower()) for v in nltk.tokenize.word_tokenize(r['text'] )])
    for t in r["textTokens"]:
        if t not in token_counts:
            token_counts[t] = 0
        token_counts[t] += 1
len(token_counts)

def find_hits(q):
    qtokens = set([ps.stem(v.lower()) for v in nltk.tokenize.word_tokenize(q)])
    print(qtokens)
    results = []
    for r in snomeds:
        matchText = r['textTokens'] & qtokens
        #print(r['tokens'] & qtokens, len(r['tokens'] & qtokens))
        if len(matchText) > 0:
            results.append([sum([10 * 60000/token_counts[t] for t in matchText]), r])
    results.sort(key=lambda v: v[0], reverse=True)
    return results[:100]


In [ ]:

for r in find_hits("major depressive disorder, recurrent, severe")[:50]:
    print(r[1]['SNOMED_CID'],r[1]['text'])#, f"(units: {r[1]['EXAMPLE_UCUM_UNITS']})")

len(snomeds)
#from txtai.embeddings import Embeddings